# Interactive Functions 

In [51]:
#!pip install voila

In [4]:
#! pip install plotnine

In [52]:
import pandas as pd
from siuba import *

import numpy as np
from plotnine import *

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive_output

In [3]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [40]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/function_data.csv', low_memory=False)

#### Dropping the duplicates

In [46]:
df.sample(5)

,Unnamed: 0,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
17570,17570,ATP,5173(037),Fowler,2020-10-07,3000.00,0.00,3389.00,Authorized,5173,6,FRE150045,Westside Of Golden State Boulevard From Adams ...,Construct A Class I Bike And Pedestrian Path,1,CFCG,2020-01-01
12299,12299,RPSTP,5270(010),Laguna Beach,2014-03-03,-39486.04,0.00,-35372.23,Authorized,NaN,12,ORA990907,Broadway (sr-133) From Acacia Drive/cliff Driv...,Landscaping And Enhanced Sidewalk,2,SCAG,2014-01-01
16146,16146,CML,5950(462),Kern County,2020-02-24,539500.00,0.00,748500.00,Authorized,5950,6,KER180513,"In Lamont Area ,wilson Ave: Hope Ave To Southe...",Paving Dirt Roads And Shldrs And Construct Cur...,1,KCOG,2020-01-01
13476,13476,SRTSLNI,5472(015),Windsor,2019-04-12,0.00,0.00,0.00,Authorized,5472,4,VAR110012,Multiple Schools In Windsor Unified School Dis...,Conduct Pedestrian And Bke Safety Education,3,MTC,2019-01-01
19102,19102,STPCML,5950(483),Kern County,2021-04-21,423977.00,0.00,505638.00,Authorized,5950,6,KER180507,"Hughes Lane, Between Terrace Way And Bakersfie...","Paving Hot Mix Asphalt, Constructing Concrete ...",1,KCOG,2021-01-01


In [47]:
(df >> count(_.project_location) >> arrange(-_.n)).head(5)

,project_location,n
6925,San Francisco Bay Area,65
8196,Various Locations,31
6846,Sacog Region,29
3028,Golden Gate Bridge,27
7864,Throughout Kern County,22


## Full Function for Prefix

In [56]:
    @interact
    
    def dla_get_prefix(place=df.agency.sort_values().unique().tolist()):
        # for the table- using one as some agencies only have one entry
        pd.set_option("display.max_columns", None)
        from IPython.core.display import display
        display(df[df.agency == place].describe()) 
        display((df[df.agency == place] >> count(_.type_of_work) >> arrange(-_.n)).head(5)) 
        # graphs 
    
        agencies = df[df.agency==place]
    
        prefix_count_n = agencies >> count(_.prefix) 
    
        ax1 = (prefix_count_n
            >> ggplot(aes("prefix", "n", fill="prefix")) 
               + geom_col() 
               + theme(axis_text_x = element_text(angle = 45 , hjust=1))
               + labs(title='Agency Program Codes', x='Program Codes', y='Number of Obligations', fill="Program Type")
        )
        return ax1

        

interactive(children=(Dropdown(description='place', options=('Access Services', 'Agoura Hills', 'Ala-Con Costa…

## Function for results by Prefix

In [49]:
@interact
def prefix_all_agencies_4(prefix=df.prefix.sort_values().unique().tolist()):
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix)].sample(1))
    
    # graphs 
    prefixes = df[df.prefix== prefix]
    #prefixes = df[df.prefix==prefix]
    
    prefix_count = (prefixes >> count(_.agency) >> arrange(-_.n)).head(5)
    
    ax1 = (prefix_count
            >> ggplot(aes("agency", "n", fill="agency")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top 5 Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1
              

interactive(children=(Dropdown(description='prefix', options=('6140000', 'ACNHPI', 'ACST-ER', 'ACSTP', 'ASCTP'…